In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("../input/ckdisease/kidney_disease.csv")

In [ ]:
df

In [ ]:
df.shape

In [ ]:
col_dict={"bp":"blood pressure",
          "sg":"specific gravity",
          "al":"albumin",
          "su":"sugar",
          "rbc":"red blood cells",
          "pc":"pus cell",
          "pcc":"pus cell clumps",
          "ba":"bacteria",
          "bgr":"blood glucose random",
          "bu":"blood urea",
          "sc":"serum creatinine",
          "sod":"sodium",
          "pot":"potassium",
          "hemo":"hemoglobin",
          "pcv":"packed cell volume",
          "wc":"white blood cell count",
          "rc":"red blood cell count",
          "htn":"hypertension",
          "dm":"diabetes mellitus",
          "cad":"coronary artery disease",
          "appet":"appetite",
          "pe":"pedal edema",
          "ane":"anemia",
          "classification":"class"}

df.rename(columns=col_dict, inplace=True)

In [ ]:
col_dict

In [ ]:
df


In [ ]:
df.dtypes

In [ ]:
#As it can be seen, red_blood_cell_count, packed_cell_volume and white_blood_cell_count are object type. We need to 
#change to numerical dtype.

In [ ]:
def convert_dtype(df,feature):
    df[feature] = pd.to_numeric(df[feature],errors='coerce')

In [ ]:
features=['red_blood_cell_count','packed_cell_volume','white_blood_cell_count']

for feature in features:
    convert_dtype(df,feature)

In [ ]:
df.dtypes

Let's drop the id column. id column is seems to be an unique identifier for each row so we are dropping that it won't help us to find any insights from the data

In [ ]:
df.drop(["id"],axis=1,inplace=True) 

# **Extract Numerical & Categorical Features**

In [ ]:
def extract_cat_num(df):
    cat_col=[col for col in df.columns if df[col].dtype=='object']
    num_col=[col for col in df.columns if df[col].dtype!='object']
    return cat_col,num_col

In [ ]:
cat_col,num_col=extract_cat_num(df)

In [ ]:
cat_col

In [ ]:
num_col

*total unique categories in our categorical features to check if any dirtiness in data or not*

In [ ]:
for col in cat_col:
    print('{} has {} values '.format(col,df[col].unique()))
    print('\n')

In [ ]:
## ckd-chronic kidney disease
## notckd-->> not crornic kidney disease

So we need to correct 2 features and the target variable which contain certain discrepancy in some values.

In [ ]:
#Replace incorrect values

df['diabetes_mellitus'].replace(to_replace = {'\tno':'no','\tyes':'yes',' yes':'yes'},inplace=True)

df['coronary_artery_disease'] = df['coronary_artery_disease'].replace(to_replace = '\tno', value='no')

df['class'] = df['class'].replace(to_replace = 'ckd\t', value = 'ckd')

In [ ]:
for col in cat_col:
    print('{} has {} values  '.format(col, df[col].unique()))
    print('\n')

In [ ]:
len(num_col)

# **Checking features distribution**

In [ ]:
plt.figure(figsize=(30,20))
for i,feature in enumerate(num_col):
    plt.subplot(5,3,i+1)
    df[feature].hist()
    plt.title(feature)

Observations:
1.     Age looks a bit left skewed
2.     Blood gluscose random is right skewed
3.     Blood Urea is also a bit right skewed
4.     Rest of the features are lightly skewed    

# **Now, let's check the label distribution for categorical data**

In [ ]:
len(cat_col)

In [ ]:
plt.figure(figsize=(20,20))
for i,feature in enumerate(cat_col):
    plt.subplot(4,3,i+1)
    sns.countplot(df[feature])

A few features have imbalanced categories. Stratified folds will be necessary while cross validation.

In [ ]:
sns.countplot(x='class',data=df)
plt.xlabel("class")
plt.ylabel("Count")
plt.title("target Class")

# **CORRELATION**

In [ ]:
plt.figure(figsize=(10,))
corr_df = df.corr()
sns.heatmap(corr_df,annot=True)

**Positive Correlation:**

* Specific gravity -> Red blood cell count, Packed cell volume and Hemoglobin
* Sugar -> Blood glucose random
* Blood Urea -> Serum creatinine
* Hemoglobin -> Red Blood cell count <- packed cell volume


**Negative Correlation:**

* Albumin, Blood urea -> Red blood cell count, packed cell volume, Hemoglobin
* Serum creatinine -> Sodium

In [ ]:
df.groupby(['red_blood_cells','class'])['red_blood_cell_count'].agg(['count','mean','median','min','max'])

In [ ]:
!pip install plotly
import plotly.express as px

In [ ]:
px.violin(df,y='red_blood_cell_count',x="class", color="class")

In [ ]:
px.scatter(df,'hemoglobin','packed_cell_volume')

In [ ]:
### analysing distribution of 'red_blood_cell_count' in both Labels 

grid=sns.FacetGrid(df, hue="class",aspect=2)
grid.map(sns.kdeplot, 'red_blood_cell_count')
grid.add_legend()

    Both distributions are quite different, distribution CKD is quite normal and evenly distributed but not CKD distribution is a little bit left-skewed but quite close to a normal distribution

In [ ]:
# Defining violin and scatter plot & kde_plot functions
def violin(col):
    fig = px.violin(df, y=col, x="class", color="class", box=True)
    return fig.show()

def scatters(col1,col2):
    fig = px.scatter(df, x=col1, y=col2, color="class")
    return fig.show()

In [ ]:
def kde_plot(feature):
    grid = sns.FacetGrid(df, hue="class",aspect=2)
    grid.map(sns.kdeplot, feature)
    grid.add_legend()

In [ ]:
kde_plot('red_blood_cell_count')

In [ ]:
kde_plot('hemoglobin')

In [ ]:
kde_plot('white_blood_cell_count')

In [ ]:
scatters('red_blood_cell_count', 'packed_cell_volume')

In [ ]:
scatters('hemoglobin','packed_cell_volume')

In [ ]:
scatters('red_blood_cell_count', 'hemoglobin')

1.RBC count range ~2 to <4.5 and Hemoglobin between 3 to <13 are mostly classified as positive for chronic kidney  
disease(i.e ckd).


2.RBC count range >4.5 to ~6.1 and Hemoglobin between >13 to 17.8 are classified as negative for chronic kidney 
disease(i.e nockd).

In [ ]:
violin('red_blood_cell_count')

In [ ]:
violin('packed_cell_volume')

# **Now let's check for negative correlation and its impact on classes**

Albumin, Blood urea -> Red blood cell count, packed cell volume, Haemoglobin

In [ ]:
scatters('red_blood_cell_count','albumin')

Clearly, albumin levels of above 0 affect ckd largely

In [ ]:
scatters('packed_cell_volume','blood_urea')

Packed cell volume >= 40 largely affects to be non ckd

In [ ]:
fig = px.bar(df, x="specific_gravity", y="packed_cell_volume",
             color='class', barmode='group',
             height=400)
fig.show()

Clearly, specific gravity >=1.02 affects non ckd

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
cat_col

In [ ]:
sns.countplot(df['red_blood_cells'])

In [ ]:
data=df.copy()

In [ ]:
### filling missing with Random value

# **Random Value Imputation¶**

In [ ]:
data['red_blood_cells'].isnull().sum()

In [ ]:
data['red_blood_cells'].dropna().sample()

In [ ]:
random_sample=data['red_blood_cells'].dropna().sample(data['red_blood_cells'].isnull().sum())
random_sample

In [ ]:
random_sample.index

In [ ]:
data[data['red_blood_cells'].isnull()].index

In [ ]:
random_sample.index=data[data['red_blood_cells'].isnull()].index

In [ ]:
random_sample.index

In [ ]:
random_sample

In [ ]:
data.loc[data['red_blood_cells'].isnull(),'red_blood_cells']=random_sample

In [ ]:
data.head()

In [ ]:
sns.countplot(data['red_blood_cells'])

In [ ]:
data['red_blood_cells'].value_counts()/len(data)

In [ ]:
len(df[df['red_blood_cells']=='normal'])/248

In [ ]:
len(df[df['red_blood_cells']=='abnormal'])/248

In [ ]:
### lets create a function so that I can easily do it for all features
def Random_value_imputation(feature):
    random_sample=data[feature].dropna().sample(data[feature].isnull().sum())               
    random_sample.index=data[data[feature].isnull()].index
    data.loc[data[feature].isnull(),feature]=random_sample



In [ ]:
Random_value_imputation('pus_cell')
Random_value_imputation('red_blood_cells')

In [ ]:
data[cat_col].isnull().sum()

In [ ]:
### as rest of the features has less missing values,so I can fill it using mode concept

In [ ]:
mode=data['pus_cell_clumps'].mode()[0]
mode

In [ ]:
data['pus_cell_clumps']=data['pus_cell_clumps'].fillna(mode)

In [ ]:
def impute_mode(feature):
    mode=data[feature].mode()[0]
    data[feature]=data[feature].fillna(mode)

In [ ]:
for col in cat_col:
    impute_mode(col)

In [ ]:
data[cat_col].isnull().sum()

In [ ]:
data[num_col].isnull().sum()

In [ ]:
### lets fill missing values in Numerical features using Random value Imputation

In [ ]:
for col in num_col:
    Random_value_imputation(col)

In [ ]:
data[num_col].isnull().sum()

**feature Encoding**

In [ ]:
for col in cat_col:
    print('{} has {} categories'.format(col, data[col].nunique()))

In [ ]:
#### as we have just 2 categories in each feature then we can consider Label Encoder as it will not cause Curse of Dimensionality

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
for col in cat_col:
    data[col]=le.fit_transform(data[col])

In [ ]:
data.head()

**Feature Importance**

In [ ]:
#SelectKBest-to select k best features
#chi2-Internally this class is going to check that whether p-value is less than 0.05 or not
#based on that,it will actually order all the features

In [ ]:
from sklearn.feature_selection import SelectKBest#Also known as Information Gain
from sklearn.feature_selection import chi2

In [ ]:
ind_col=[col for col in data.columns if col!='class']
dep_col='class'

In [ ]:
X=data[ind_col]
y=data[dep_col]

In [ ]:
ordered_rank_features=SelectKBest(score_func=chi2,k=20)
ordered_feature=ordered_rank_features.fit(X,y)

In [ ]:
ordered_feature

In [ ]:
#To get scores(rank) of feature,what we can do we can use scores function
ordered_feature.scores_

In [ ]:
datascores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])
datascores

In [ ]:
dfcolumns=pd.DataFrame(X.columns)
dfcolumns

In [ ]:
features_rank=pd.concat([dfcolumns,datascores],axis=1)

In [ ]:
features_rank

In [ ]:
#Higher the score is,more important feature is 
features_rank.columns=['Features','Score']
features_rank

In [ ]:
#fetch largest 10 values of Score column
features_rank.nlargest(10,'Score')

In [ ]:
selected_columns=features_rank.nlargest(10,'Score')['Features'].values

In [ ]:
X_new=data[selected_columns]

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new,y,train_size=0.75)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
## check whether dataset is imbalance or not
y_train.value_counts()

In [ ]:
### Lets find best model using Hyperparameter optimization

In [ ]:
!pip install xgboost
from xgboost import XGBClassifier
XGBClassifier()

In [ ]:
## Hyper Parameter Optimization with respect to XGBoost

params={
 "learning_rate"    : [0.05, 0.20, 0.25 ] ,
 "max_depth"        : [ 5, 8, 10, 12],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.7 ]
    
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from xgboost import XGBClassifier
classifier=XGBClassifier()

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:
classifier=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=5,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

## we have got this model on the basis of cross valudation & hyper-parameter optimization

In [ ]:
classifier.fit(X_train,y_train)

In [ ]:
y_pred=classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

In [ ]:
### to make confusion matrix user-friendly
plt.imshow(confusion)

In [ ]:
accuracy_score(y_test, y_pred)